Using the job details of Enron's executives, we want to determine who is likely to be a Person Of Interest (POI).
The status of POI (1) or non POI (0) has been manually determined from court investigation files.

In [371]:
import sys
import pickle
import numpy
sys.path.append("../../../GitHub/ud120-projects/tools/")
from feature_format import featureFormat, targetFeatureSplit

Load the dataset:

In [372]:
data_dict = pickle.load(open("../../../GitHub/ud120-projects/final_project/final_project_dataset.pkl", "r") )
del data_dict["TOTAL"] # Remove TOTAL row outlier
print "Number of people in the dataset:",len(data_dict)

Number of people in the dataset: 145


Let's show the list of features that we have for each Enron executive by selecting one at random:

In [373]:
features_original = data_dict['METTS MARK'].keys()
print features_original

['salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'email_address', 'from_poi_to_this_person']


(Notice that the label to be predicted is the feature 'poi')

Create a set of new features derived from those in the dataset, which I believe could provide useful insight on illegal activities:

In [374]:
for person in data_dict:
    poi_from   = data_dict[person]['from_this_person_to_poi']
    poi_to   = data_dict[person]['from_poi_to_this_person']
    total_from = data_dict[person]['from_messages']
    total_to = data_dict[person]['to_messages']

    if total_from != 0:
        data_dict[person]['fraction_from_poi'] = float(poi_from)/float(total_from)
    else:
        data_dict[person]['fraction_from_poi'] = float(total_from)

    if total_to != 0:
        data_dict[person]['fraction_to_poi'] = float(poi_to)/float(total_to)
    else:
        data_dict[person]['fraction_to_poi'] = float(total_to)

    if poi_from != 0:
        data_dict[person]['poi_emails_ratio'] = float(poi_to)/float(poi_from)
    else:
        data_dict[person]['poi_emails_ratio'] = numpy.NaN

    if total_from != 0.:
        data_dict[person]['total_emails_ratio'] = float(total_to)/float(total_from)
    else:
        data_dict[person]['total_emails_ratio'] = numpy.NaN

The list of features I will to investigate into:

In [375]:
features_list = [ 'poi', 'salary', 'bonus', 'from_this_person_to_poi', 'from_poi_to_this_person', 
                  'fraction_from_poi', 'fraction_to_poi', 'total_emails_ratio', 'shared_receipt_with_poi',
                  'director_fees', 'deferral_payments', 'poi_emails_ratio' ]
features_name = features_list[1:]

Let's first divide the dataset beetween features and our target label:

In [376]:
my_dataset = featureFormat(data_dict,features_list)
poi, features = targetFeatureSplit( my_dataset )

The number of executives whose activity details are contained in our dataset is:

In [377]:
print "Number of people in the dataset:",len(features) 

Number of people in the dataset: 145


Out of which, we have the following number of POIs:

In [378]:
print "Number of POIs in the dataset:",sum(poi)," (",sum(poi)/float(len(features))*100.,"% )"

Number of POIs in the dataset: 18.0  ( 12.4137931034 % )


Let's plot some of these features against each other to have a feeling of how well-separated are our target labels.

In [405]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import gridspec

# Regroup every feature in a single list, one value for each person
features_split = zip(*features)

plot_combinations = [ ('salary','bonus'), ('fraction_from_poi','bonus'), 
                      ('fraction_from_poi','shared_receipt_with_poi'),
                      ('from_poi_to_this_person','fraction_from_poi'),
                      ('total_emails_ratio','from_poi_to_this_person'),
                      ('salary','shared_receipt_with_poi')]

fig = plt.figure(figsize=(16, 10))
gs  = gridspec.GridSpec(2, 3) 

for i,(feat1,feat2) in enumerate(plot_combinations):
    index1   = features_name.index(feat1)
    index2   = features_name.index(feat2)
    feature1 = features_split[index1]
    feature2 = features_split[index2]
    feature1 = numpy.reshape( numpy.array(feature1), (len(feature1), 1))
    feature2 = numpy.reshape( numpy.array(feature2), (len(feature2), 1))
    plt.subplot(gs[i]).scatter(feature1,feature2,color="b")
    
    for ii, pp in enumerate(feature1):
        if poi[ii]:
            plt.scatter(feature1[ii], feature2[ii], color="r")

    plt.xlabel(feat1)
    plt.ylabel(feat2)

    # Show a legend
    red_points  = mpatches.Patch(color='red',  label='POI')
    blue_points = mpatches.Patch(color='blue', label='non POI')
    if i in [0,1,3,4]:
        plt.yscale('log')
    if i in [1,2,3,4]:
        plt.xscale('log')
    if i in [0]:
        plt.subplot(gs[i]).legend(handles=[blue_points,red_points],loc='lower right')
    elif i in [1,3]:
        plt.subplot(gs[i]).legend(handles=[blue_points,red_points],loc='lower left')
    else:
        plt.subplot(gs[i]).legend(handles=[blue_points,red_points],loc='best')

plt.tight_layout()
plt.show()

Some of the features we are interested in are empty for certain executives. This is the number of Enron executives from which we don't know each feature in the list:

In [406]:
print "NaN entries for each feature:"
for (name,feat) in zip(features_name,features_split):
    print name,":",sum(numpy.isnan(feat))

NaN entries for each feature:
salary : 0
bonus : 0
from_this_person_to_poi : 0
from_poi_to_this_person : 0
fraction_from_poi : 59
fraction_to_poi : 59
total_emails_ratio : 59
shared_receipt_with_poi : 0
director_fees : 0
deferral_payments : 0
poi_emails_ratio : 79


To fix this problem, we fill the empty fields with the mean value of the corresponding feature:

In [381]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
features_filled = imp.fit_transform(features)

I would like to test several classification algorithm, but the performance of some of them such as SVC will be non-optimal given that the typical values of each feature can differ by several orders of magnitude. We perform a lineal rescaling to fix this problem. Algotithm that don't need this rescaling won't be affected either positively or negatively.  

In [382]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features_rescaled = scaler.fit_transform(features_filled)

We will now divide our dataset into a 66.7% training sample and a 33.3% testing sample to measure the final performance of our algorithm:

In [383]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(poi,3)
i = 0
for train, test in skf:
    features_train = [ features_rescaled[ii] for ii in train ] 
    features_test  = [ features_rescaled[ii] for ii in test ]
    labels_train   = [ poi[ii] for ii in train ] 
    labels_test    = [ poi[ii] for ii in test ] 
    if i == 1: 
        break
    i += 1


I have used StratifiedKFold so there is the same fraction of positive datapoints both in our test and training datasets (~12%). Let's check that:

In [407]:
print "Fraction of POIs in the test and train samples:",
print float(sum(labels_train))/float(len(labels_train))*100.,"% , ",
print float(sum(labels_test))/float(len(labels_test))*100.,"%"
print sum(labels_test)

Fraction of POIs in the test and train samples: 12.3711340206 % ,  12.5 %
6.0


We will now select the 5 most important features for our dataset, which are those with the highest score on a SelectKBest algorithm:

In [408]:
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(k=6)
X_train = selector.fit_transform(features_train, labels_train)
X_test  = selector.transform(features_test)

feat_score = selector.scores_.tolist()
feat_pval = selector.pvalues_.tolist()

for fs,name,fp in sorted(zip(feat_score,features_name,feat_pval),reverse=True):
    print "%s score: %.2f ( p = %.1e )" % (name,fs,fp)

fraction_from_poi score: 7.98 ( p = 5.8e-03 )
salary score: 7.06 ( p = 9.2e-03 )
bonus score: 3.78 ( p = 5.5e-02 )
from_this_person_to_poi score: 2.03 ( p = 1.6e-01 )
shared_receipt_with_poi score: 1.76 ( p = 1.9e-01 )
director_fees score: 1.31 ( p = 2.6e-01 )
deferral_payments score: 1.13 ( p = 2.9e-01 )
poi_emails_ratio score: 0.56 ( p = 4.6e-01 )
from_poi_to_this_person score: 0.56 ( p = 4.6e-01 )
fraction_to_poi score: 0.51 ( p = 4.8e-01 )
total_emails_ratio score: 0.14 ( p = 7.1e-01 )


In [386]:
feat_support = selector.get_support().tolist()
feat_selected = []
print "Selected indexes:",
for f in [ fl for (fl,fs) in zip(features_name,feat_support) if fs ]:
    print f,",",
    feat_selected.append(f)

Selected indexes: salary , bonus , from_this_person_to_poi , fraction_from_poi , shared_receipt_with_poi , director_fees ,


We will use these 6 features to predict who is and who is not a POI. 

Since our fraction of positive labels (POIs) is pretty low, our evaluation of the algorithm performance can not be based on its accuracy. Let's select a proper evaluation metric that fits the intended result interpretation:

I would like to help the investigation by pinning down possible suspects for a closer police investigation, so my algorithm finds all the possible POIs even if I have to pay the price of a few false positives, i.e. accusing innocent people. Algorithms are not providing the final veredict about these executives' fraud activities, and so those innocent Enron employees will be discarded later on after a deeper fiscal investigation.

Therefore, we will optimize our algorithm for a maximum recall.

In [387]:
from sklearn.metrics import recall_score
score = 'recall'
best_performance = { }
best_algorithm = { }

We will now test the performance of the following classification algorithms:

In [388]:
classifier_names = [ 'GaussianNB' , 'DecisionTreeClassifier', 'LogisticRegression', 'SVC' ]

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

classifiers      = [ GaussianNB() , DecisionTreeClassifier(), LogisticRegression(), SVC() ]

We will test each of these algorithms, as well as several combination of tunning parameters, using a cross-validated grid search:

In [397]:
from sklearn.grid_search import GridSearchCV

param_grid = [ {  },
               { 'criterion':         [ 'gini' , 'entropy' ],
                 'max_depth':         [ 15, 8 , 6 , 4 ] ,
                 'min_samples_split': [ 10 , 6 , 4 , 2] } ,
               { 'C': [ 1000 , 100 , 10 , 5 , 1 ] } ,
               { 'kernel': [ 'linear', 'rbf', 'sigmoid'] ,
                 'C':      [ 1000, 100, 10, 5, 1] ,
                 'gamma': [ 0.5 , 1.0 , 10. ] }
             ]

In [409]:
max_score = 0.

for index,classifier in enumerate(classifiers):
    print ">>> %s :: Tuning parameters for %s" % (classifier_names[index],score)
    clf = GridSearchCV(classifier,param_grid[index],scoring="%s"%(score),cv=10)

    clf.fit(X_train,labels_train)

    print " Best parameters set found on training set:",
    print clf.best_params_
    print "",score,"scores:"
    for params, mean_score, cv_scores in clf.grid_scores_:
        print "%0.3f (+/-%0.03f) for %r" % (mean_score, cv_scores.std() * 2, params)
    print ""
    best_performance[classifier_names[index]] = "%0.3f with parameters %r" % (clf.best_score_, clf.best_params_)
    
    if clf.best_score_ > max_score:
        max_score = clf.best_score_
        best_algorithm[max_score] = (classifier,clf.best_params_)

>>> GaussianNB :: Tuning parameters for recall
 Best parameters set found on training set: {}
 recall scores:
0.794 (+/-0.800) for {}

>>> DecisionTreeClassifier :: Tuning parameters for recall
 Best parameters set found on training set: {'min_samples_split': 2, 'criterion': 'gini', 'max_depth': 8}
 recall scores:
0.263 (+/-0.806) for {'min_samples_split': 10, 'criterion': 'gini', 'max_depth': 15}
0.206 (+/-0.800) for {'min_samples_split': 6, 'criterion': 'gini', 'max_depth': 15}
0.206 (+/-0.800) for {'min_samples_split': 4, 'criterion': 'gini', 'max_depth': 15}
0.263 (+/-0.806) for {'min_samples_split': 2, 'criterion': 'gini', 'max_depth': 15}
0.206 (+/-0.800) for {'min_samples_split': 10, 'criterion': 'gini', 'max_depth': 8}
0.263 (+/-0.806) for {'min_samples_split': 6, 'criterion': 'gini', 'max_depth': 8}
0.206 (+/-0.800) for {'min_samples_split': 4, 'criterion': 'gini', 'max_depth': 8}
0.299 (+/-0.917) for {'min_samples_split': 2, 'criterion': 'gini', 'max_depth': 8}
0.263 (+/-0.80

Let's print the maximum recall obtained by each of the algorithm, together with the optimum tunning parameters:

In [410]:
print "For optimum",score,"in our training set:"
for name in classifier_names:
    print "  ",name,":  ",best_performance[name]

For optimum recall in our training set:
   GaussianNB :   0.794 with parameters {}
   DecisionTreeClassifier :   0.299 with parameters {'min_samples_split': 2, 'criterion': 'gini', 'max_depth': 8}
   LogisticRegression :   0.000 with parameters {'C': 1000}
   SVC :   0.320 with parameters {'kernel': 'sigmoid', 'C': 1000, 'gamma': 10.0}


We can see that we have obtained similar performance with the 4 classifiers but a maximum recall of 79.4% has been obtained using a simple Gaussian Naive Bayes classifier. Let's now measure the performance of this algorithm using the held-out test subsample:

In [411]:
from time import time
from sklearn.metrics import classification_report,confusion_matrix

selected_classifier = best_algorithm[max_score][0]
selected_parameters = best_algorithm[max_score][1]

print "Predicting POIs on the test set using a",classifier_names[classifiers.index(selected_classifier)],"classifier"
print "Parameters:",selected_parameters
t0 = time()
clf = selected_classifier.set_params(**selected_parameters)
clf.fit(X_train,labels_train)
labels_pred = clf.predict(X_test)
print "Predictions obtained in %0.3fs" % (time() - t0)

print classification_report(labels_test, labels_pred)

print "\t\t      Predicted"
print "\t\t non-POI\tPOI"
for ii,row in enumerate(confusion_matrix(labels_test, labels_pred, labels=(0,1))):
    if ii == 0:
        print "True\tnon-POI: ",
    if ii == 1:
        print "\tPOI:\t  ",
    for element in row:
        print element,"\t\t",
    print ""
    
print "Recall:",recall_score(labels_test,labels_pred)*100.,"%"

Predicting POIs on the test set using a GaussianNB classifier
Parameters: {}
Predictions obtained in 0.001s
             precision    recall  f1-score   support

        0.0       1.00      0.14      0.25        42
        1.0       0.14      1.00      0.25         6

avg / total       0.89      0.25      0.25        48

		      Predicted
		 non-POI	POI
True	non-POI:  6 		36 		
	POI:	   0 		6 		
Recall: 100.0 %
